1 - get the nodes state observations : their informations through the rounds + the accuracy of their models

In [1]:
import json

# Load the JSON data from the file
with open('../../subdir/results/scenario1/score_alpha02_nodes50/blockchain.json', 'r') as f:
    data = json.load(f)

# Create a list to store node information for each block
node_info_by_block = {}
transitions = []
nodes_by_block=[]
# transition = {"obs" : , "obs_" , "action" : , "reward" : ,"done" : }
# Iterate through the transactions of all blocks
for json_obj in data:
    block_id = json_obj['BlockId']

    for transaction in json_obj['Transactions']:
        content = transaction.get('Content', {})
        message_type = content.get('message_type')
        node_id = content.get('nodeId')

        if message_type == 1 and node_id is not None:
            availability = int(content.get('availability'))
            honesty = content.get('honesty')
            dataset_size = content.get('datasetSize')
            freq = content.get('freq')
            trans_rate = content.get('transRate')
            task = content.get('task')
            dropout = content.get('dropout', 0.0)

            # Find accuracy for the corresponding node in message_type 2 transactions
            accuracy = 0.0
            for json_obj2 in data:
                if json_obj2['BlockId'] == block_id:
                    for transaction2 in json_obj2['Transactions']:
                        if transaction2['Content'].get('message_type') == 2 and transaction2['Content'].get('nodeId') == node_id and transaction2['Content'].get('type') == 0:
                            accuracy = transaction2['Content'].get('accuracy', 0.0)
                            break
                    if accuracy != 0.0:
                        break

            node_info = [node_id, availability, honesty, dataset_size, freq, trans_rate, dropout, accuracy]

            if block_id not in node_info_by_block:
                node_info_by_block[block_id] = []

            node_info_by_block[block_id].append(node_info)

# Sort the node lists within each block by nodeId
for nodes in node_info_by_block.values():
    nodes.sort(key=lambda x: x[0])  # Sort based on nodeId

# Print or use node_info_by_block as needed
for block_id, nodes in node_info_by_block.items():
    print(f"Block {block_id}:")
    nodes_by_block.append(nodes)
    for node_info in nodes:
        print(node_info)
   


Block 1/0:
[0, 1, 0.0, 320, 190.0, 260.0, 0.0, 0.0]
[1, 1, 0.0, 360, 290.0, 770.0, 0.0, 0.0]
[2, 1, 0.0, 510, 100.0, 730.0, 0.0, 0.0]
[3, 1, 0.0, 450, 180.0, 450.0, 0.0, 56.67]
[4, 1, 0.0, 800, 280.0, 640.0, 0.0, 68.75]
[5, 1, 0.0, 130, 190.0, 270.0, 0.0, 0.0]
[6, 1, 0.0, 760, 240.0, 830.0, 0.0, 69.74]
[7, 1, 0.0, 160, 120.0, 670.0, 0.0, 0.0]
[8, 0, 0.0, 380, 260.0, 540.0, 0.0, 0.0]
[9, 1, 0.0, 300, 60.0, 990.0, 0.0, 0.0]
[10, 1, 0.0, 610, 50.0, 700.0, 0.0, 0.0]
[11, 1, 0.0, 410, 250.0, 180.0, 0.0, 0.0]
[12, 0, 0.0, 520, 200.0, 460.0, 0.0, 0.0]
[13, 1, 0.0, 840, 300.0, 750.0, 0.0, 0.0]
[14, 1, 0.0, 700, 180.0, 890.0, 0.0, 72.86]
[15, 1, 0.0, 830, 270.0, 470.0, 0.0, 63.25]
[16, 1, 0.0, 450, 150.0, 550.0, 0.0, 63.33]
[17, 0, 0.0, 460, 70.0, 360.0, 0.0, 0.0]
[18, 1, 0.0, 650, 160.0, 240.0, 0.0, 0.0]
[19, 0, 0.0, 550, 240.0, 970.0, 0.0, 0.0]
[20, 1, 0.0, 210, 70.0, 160.0, 0.0, 0.0]
[21, 1, 0.0, 580, 290.0, 180.0, 0.0, 0.0]
[22, 1, 0.0, 500, 220.0, 190.0, 0.0, 0.0]
[23, 1, 0.0, 700, 160.0, 

2 - get the global accuracy for each round

In [2]:
# Create a list to store the global accuracy values
global_accuracy = []

# Iterate through the transactions of all blocks
for json_obj in data:
    block_id = json_obj['BlockId']
    
    # Find the transaction with message_type equal to 2 and type equal to 2
    accuracy_transaction = None
    for transaction in json_obj['Transactions']:
        content = transaction['Content']
        if content.get('message_type') == 2 and content.get('type') == 2:
            accuracy_transaction = transaction
            break
    
    if accuracy_transaction is not None:
        accuracy = accuracy_transaction['Content'].get('accuracy', 0.0)
        global_accuracy.append(accuracy)

# Print or use global_accuracy as needed
print("Global Accuracy Values:", global_accuracy)

Global Accuracy Values: [60.0, 73.53, 74.71, 83.0, 83.0, 85.0, 86.0, 87.0, 85.0, 90.0, 88.0, 89.0, 87.0, 89.69, 89.84, 92.0, 92.0, 93.0, 93.0, 91.88, 92.0, 93.0, 94.24, 91.21, 93.27, 93.0, 91.76, 91.0, 93.45]


In [3]:
# Create a list to store the concatenated lists for each block
selected_nodes = []

# Iterate through the transactions of all blocks
for json_obj in data:
    block_id = json_obj['BlockId']
    
    # Find the transaction with message_type equal to 4
    aggregator_transaction = None
    for transaction in json_obj['Transactions']:
        if transaction['Content'].get('message_type') == 4:
            aggregator_transaction = transaction
            break
    
    if aggregator_transaction is not None:
        aggregators = aggregator_transaction['Content'].get('aggregators', [])
        trainers = aggregator_transaction['Content'].get('trainers', [])
        
        selected_nodes.append(aggregators + trainers)

# Print or use selected_nodes as needed
for block_id, selected_node_list in zip([json_obj['BlockId'] for json_obj in data], selected_nodes):
    print(f"Block {block_id}: {selected_node_list}")

Block 1/0: [32, 1, 7, 30, 35, 13, 42, 6, 4, 27, 14, 36, 16, 38, 3, 15, 23]
Block 1/1: [37, 1, 7, 30, 35, 46, 19, 8, 13, 42, 43, 6, 4, 49, 27, 36, 16]
Block 1/2: [37, 30, 35, 19, 42, 49, 27, 1, 36, 16, 7, 38, 3, 12, 15, 23, 9]
Block 1/3: [32, 37, 30, 35, 19, 42, 13, 43, 27, 4, 36, 12, 15, 23, 9, 26, 5]
Block 1/4: [32, 37, 30, 35, 19, 42, 46, 13, 43, 27, 49, 4, 36, 16, 38, 12, 15]
Block 1/5: [32, 30, 19, 46, 13, 43, 42, 49, 36, 16, 38, 15, 26, 29, 2, 45, 48]
Block 1/6: [32, 30, 19, 35, 46, 13, 42, 49, 27, 38, 15, 23, 26, 29, 45, 48, 6]
Block 1/7: [32, 30, 37, 19, 35, 46, 13, 42, 43, 49, 27, 16, 38, 15, 23, 26, 29]
Block 1/8: [32, 30, 37, 19, 35, 46, 13, 42, 43, 49, 27, 36, 16, 38, 15, 12, 23]
Block 1/9: [32, 37, 30, 19, 35, 46, 13, 42, 43, 27, 36, 16, 38, 15, 12, 23, 26]
Block 1/10: [32, 37, 30, 19, 35, 46, 13, 43, 27, 16, 38, 15, 12, 23, 26, 29, 45]
Block 1/11: [32, 37, 30, 46, 13, 43, 42, 49, 16, 38, 36, 12, 23, 26, 29, 45, 48]
Block 1/12: [32, 30, 19, 46, 13, 42, 27, 16, 38, 36, 15, 1

In [4]:
round_reward = []
transitions = []

for block_id in range(len(node_info_by_block) - 1):  # Fixed the range
    # print(len(node_info_by_block) - 1)
    # print(f"Block {block_id}:")
    nodes_rewards = []
    
    for node_index in selected_nodes[block_id]:
        # print("Node index: ", node_index)
        node_data = nodes_by_block[block_id][node_index]
        reward = node_data[-1]
        # print("Original reward: ", reward)
        
        if reward != 0.0:
            reward *= node_data[2]
        else :
            reward = node_data[2]
        # print("Modified reward: ", reward)
        
        nodes_rewards.append(reward)
    
    # print("Nodes rewards:", nodes_rewards)
    cumulated_reward = sum(nodes_rewards)
    round_reward.append(cumulated_reward)
    
    transition = {
        "obs": nodes_by_block[block_id],
        "obs_": nodes_by_block[block_id + 1],
        "action": selected_nodes[block_id],
        "reward": cumulated_reward,
        "done": block_id == len(node_info_by_block) - 1  # Corrected the condition
    }
    transitions.append(transition)

for transition in transitions : 
    print(transition)


{'obs': [[0, 1, 0.0, 320, 190.0, 260.0, 0.0, 0.0], [1, 1, 0.0, 360, 290.0, 770.0, 0.0, 0.0], [2, 1, 0.0, 510, 100.0, 730.0, 0.0, 0.0], [3, 1, 0.0, 450, 180.0, 450.0, 0.0, 56.67], [4, 1, 0.0, 800, 280.0, 640.0, 0.0, 68.75], [5, 1, 0.0, 130, 190.0, 270.0, 0.0, 0.0], [6, 1, 0.0, 760, 240.0, 830.0, 0.0, 69.74], [7, 1, 0.0, 160, 120.0, 670.0, 0.0, 0.0], [8, 0, 0.0, 380, 260.0, 540.0, 0.0, 0.0], [9, 1, 0.0, 300, 60.0, 990.0, 0.0, 0.0], [10, 1, 0.0, 610, 50.0, 700.0, 0.0, 0.0], [11, 1, 0.0, 410, 250.0, 180.0, 0.0, 0.0], [12, 0, 0.0, 520, 200.0, 460.0, 0.0, 0.0], [13, 1, 0.0, 840, 300.0, 750.0, 0.0, 0.0], [14, 1, 0.0, 700, 180.0, 890.0, 0.0, 72.86], [15, 1, 0.0, 830, 270.0, 470.0, 0.0, 63.25], [16, 1, 0.0, 450, 150.0, 550.0, 0.0, 63.33], [17, 0, 0.0, 460, 70.0, 360.0, 0.0, 0.0], [18, 1, 0.0, 650, 160.0, 240.0, 0.0, 0.0], [19, 0, 0.0, 550, 240.0, 970.0, 0.0, 0.0], [20, 1, 0.0, 210, 70.0, 160.0, 0.0, 0.0], [21, 1, 0.0, 580, 290.0, 180.0, 0.0, 0.0], [22, 1, 0.0, 500, 220.0, 190.0, 0.0, 0.0], [23,

In [ ]:
import sys
# sys.path.insert(0, '../')  # Add the directory containing "DRL" to the path

from agent import node_selection_agent as ag
from environment import node_selection_env as nds

total_nodes = 50
max_actions = 17
num_features = 8
target_accuracy = 99
max_rounds= 30
#create the nodes selection environment accoding to the observation
envNodeSelect = nds.FLNodeSelectionEnv(total_nodes,max_actions, num_features=num_features, target=target_accuracy, max_rounds=max_rounds)
obs_shape = envNodeSelect.observation_space.sample()["current_state"].shape
print(obs_shape)
agent = ag.Agent(input_shape=obs_shape ,n_actions=obs_shape[0], env=envNodeSelect,max_actions=max_actions) 

# train the agent with the transitions store the trasitions then make the agent learn
for transition in transitions : 
    agent.remember(transition["obs"],transition["action"], transition["reward"], transition["obs_"], transition["done"])
    agent.learn()

: 

In [8]:
from collections import defaultdict


# Create a dictionary to store the concatenated lists for each block
selected_nodes = defaultdict(list)

# Iterate through the transactions of all blocks
for json_obj in data:
    block_id = json_obj['BlockId']
    
    # Find the transaction with message_type equal to 4
    aggregator_transaction = None
    for transaction in json_obj['Transactions']:
        if transaction['Content'].get('message_type') == 4:
            aggregator_transaction = transaction
            break
    
    if aggregator_transaction is not None:
        aggregators = aggregator_transaction['Content'].get('aggregators', [])
        trainers = aggregator_transaction['Content'].get('trainers', [])
        
        selected_nodes[block_id] = aggregators + trainers

# Count how many times each node was selected
node_selection_counts = defaultdict(int)
for selected_node_list in selected_nodes.values():
    for node_id in selected_node_list:
        node_selection_counts[node_id] += 1

# Print or use node_selection_counts as needed
for node_id, count in node_selection_counts.items():
    print(f"Node {node_id} was selected {count} times.")


Node 35 was selected 3 times.
Node 82 was selected 2 times.
Node 79 was selected 3 times.
Node 17 was selected 3 times.
Node 48 was selected 2 times.
Node 49 was selected 3 times.
Node 88 was selected 2 times.
Node 30 was selected 3 times.
Node 15 was selected 4 times.
Node 90 was selected 4 times.
Node 65 was selected 3 times.
Node 83 was selected 3 times.
Node 57 was selected 2 times.
Node 3 was selected 4 times.
Node 1 was selected 4 times.
Node 0 was selected 3 times.
Node 54 was selected 4 times.
Node 97 was selected 4 times.
Node 85 was selected 4 times.
Node 81 was selected 4 times.
Node 21 was selected 4 times.
Node 93 was selected 1 times.
Node 56 was selected 3 times.
Node 64 was selected 4 times.
Node 2 was selected 2 times.
Node 41 was selected 4 times.
Node 60 was selected 2 times.
Node 91 was selected 3 times.
Node 22 was selected 3 times.
Node 55 was selected 3 times.
Node 74 was selected 4 times.
Node 33 was selected 2 times.
Node 62 was selected 1 times.
Node 94 was se